# OpenAI Agents

In [1]:
!pip install -Uq openai-agents

In [2]:
# to use non-openai models, e.g., from Hugging Face
!pip install -Uq "openai-agents[litellm]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 105.7 MB/s eta 0:00:0000:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 20.9 MB/s eta 0:00:00


In [3]:
# enable async in notebook
import nest_asyncio

nest_asyncio.apply()

In [5]:
import os
import getpass

# set default model for agents
os.environ["OPENAI_DEFAULT_MODEL"] = "gpt-5-mini"

# openai API key
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

## Quickstart

In [6]:
from agents import Agent, function_tool, Runner

@function_tool
def get_weather(city: str) -> str:
    """returns weather info for the specified city."""
    return f"The weather in {city} is sunny"

agent = Agent(
    name="Haiku agent",
    instructions="Always respond in haiku form",
    model="gpt-5-mini",
    tools=[get_weather],
)

result = await Runner.run(agent, "What's the weather in New York?")

print(result.final_output)

Clear sun warms the streets,  
Blue sky over bustling blocks —  
New York basks in light.


In [ ]:
print(type(result))

from pprint import pprint
pprint(result.__dict__, indent=1)

## Non Open-AI models

To use Hugging Face models with OpenAI Agents SDK, you need to set the `HF_TOKEN` env variable.

Then set the model param to:

```
litellm/huggingface/<provider>/<hf_org_or_user>/<hf_model>
```

If you prefer to have more control, you can use the `LitellmModel` class.

In [19]:
import getpass

os.environ["HF_TOKEN"] = getpass.getpass("Enter your Hugging Face token: ")

In [32]:
# using kimi-k2-thinking

from agents import Agent, Runner, ModelSettings
from agents.extensions.models.litellm_model import LitellmModel

hf_model = LitellmModel(
    model="huggingface/nscale/Qwen/Qwen3-8B",
    api_key=os.environ["HF_TOKEN"],
)

hf_agent = Agent(
    name="Kimi agent",
    instructions="Always respond in haiku form",
    tools=[get_weather],
    # model="litellm/huggingface/nscale/Qwen/Qwen3-8B",
    model=hf_model,
    
    # optional, for usage tracking (requires openai API key)
    model_settings=ModelSettings(include_usage=True,),
)

result = await Runner.run(hf_agent, "What's the weather in New York?")

/usr/local/lib/python3.12/dist-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='\n\n', r...nside the XML tags.\n'}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='to...s={'stop_reason': None}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
/usr/local/lib/python3.12/dist-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='\n\nNew ...e weather response.\n'}), input_type=Message])
  PydanticSerializationUnexpectedValue(Exp

In [26]:
print(result.final_output)



New York shines today  
Sky stretches wide, bright and clear  
Breeze carries cheer


In [ ]:
pprint(result.__dict__)

## Agents

### Tracing

You can trace the execution of the agent on the OpenAI platform. To do this, you need to set an OpenAI API key like this:

In [ ]:
import os
from agents import set_tracing_export_api_key, Agent, Runner
from agents.extensions.models.litellm_model import LitellmModel

tracing_api_key = os.environ["OPENAI_API_KEY"]
set_tracing_export_api_key(tracing_api_key)

Or you can pass the API key as a parameter to the Runner. Like this:

In [ ]:
from agents import Runner, RunConfig

await Runner.run(
    agent,
    input="Hello",
    run_config=RunConfig(tracing={"api_key": "sk-..."}),
)

### Structured Output

- Use the `output_type` param with Pydantic objects (or dataclasses, lists, TypedDicts, etc.)
- When using non-openai models, remember to check that they support both Structured Ouput AND tool calling.

In [27]:
from pydantic import BaseModel
from agents import Agent
from agents.extensions.models.litellm_model import LitellmModel
import os

if "HF_TOKEN" not in os.environ:
    os.environ["HF_TOKEN"] = input("Enter your Hugging Face token: ")

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

model = LitellmModel(
    model="huggingface/novita/moonshotai/Kimi-K2-Instruct-0905",
    api_key=os.environ["HF_TOKEN"],
)

agent = Agent(
    name="Calendar extractor",
    instructions="Extract calendar events from the text",
    output_type=CalendarEvent,
    model=model,
    # model_settings=ModelSettings(include_usage=True),
)

email = """
Hello, 

I just wanted to confirm the appointment with Dr. Drake Ramoray for Wednesday at 5 p.m. 

Please let me know if there is any additional information that you need. 

Best, John 
"""

result = await Runner.run(
    agent,
    f"Extract the event from the following text: {email}",
)

print(result.final_output)

name='Appointment with Dr. Drake Ramoray' date='Wednesday' participants=['Dr. Drake Ramoray', 'John']


/usr/local/lib/python3.12/dist-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='{\n  "na...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


### Multi-Agent systems

Two main architectures:
- Manager (agents as tools): A central manager/orchestrator invokes specialized sub‑agents as tools and retains control of the conversation.
- Handoffs: Peer agents hand off control to a specialized agent that takes over the conversation. This is decentralized.


#### Agents Handoffs

In [28]:
from agents import Agent

glm_model = LitellmModel(
    model="huggingface/novita/zai-org/GLM-4.7",
    api_key=os.environ["HF_TOKEN"],
)

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
    model=glm_model,
    model_settings=ModelSettings(include_usage=True),
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=glm_model,
    model_settings=ModelSettings(include_usage=True),
)

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent],
    model=glm_model,
    model_settings=ModelSettings(include_usage=True),
)

result = await Runner.run(
    triage_agent,
    "Can you explain the causes of World War II?",
)

print(result.final_output)

/usr/local/lib/python3.12/dist-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content="I'll tra... History Tutor agent.'}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='to...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
/usr/local/lib/python3.12/dist-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='The caus...ists for readability."}), input_type=Message])
  PydanticSerializationUnexpectedValue(Exp

The causes of World War II are complex and rooted in the unresolved issues of World War I, the rise of totalitarian regimes, and the failure of international diplomacy. Here is a clear breakdown of the key factors that led to the outbreak of the war in 1939.

### 1. The Treaty of Versailles (1919)
After World War I, the Treaty of Versailles imposed harsh penalties on Germany:
*   **War Guilt Clause:** Germany was forced to accept full responsibility for the war.
*   **Reparations:** Germany had to pay massive financial reparations to the Allies, which crippled its economy.
*   **Territorial Losses:** Germany lost significant territory (e.g., Alsace-Lorraine to France, the Polish Corridor) and all its overseas colonies.
*   **Military Restrictions:** The German military was severely limited in size and capability.

These measures created deep resentment and humiliation in Germany, creating a fertile ground for radical political movements.

### 2. The Rise of Fascism and Totalitarianism


#### Agents As Tools

In [29]:
manager_agent = Agent(
    name="Manager Agent",
    instructions="You manage a team of agents to answer user questions effectively.",
    tools=[
        history_tutor_agent.as_tool(
            tool_name="history_tutor_agent",
            tool_description="Handles historical queries",
        ),
        math_tutor_agent.as_tool(
            tool_name="math_tutor_agent",
            tool_description="Handles math questions",
        ),
    ],
    model=glm_model,
    model_settings=ModelSettings(include_usage=True),
)

result = await Runner.run(
    manager_agent,
    "Can you explain the causes of World War II?",
)

print(result.final_output)

/usr/local/lib/python3.12/dist-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='', role=...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='to...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
/usr/local/lib/python3.12/dist-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='World Wa...vided good response).'}), input_type=Message])
  PydanticSerializationUnexpectedValue(Exp

The causes of World War II were complex and interconnected. Here's a comprehensive explanation:

## The Main Causes of World War II

### 1. The Legacy of World War I and the Treaty of Versailles
The Treaty of Versailles (1919) that ended World War I imposed harsh terms on Germany:
- Germany was forced to accept full blame for the war
- Required to pay massive reparations
- Lost significant territory (including Alsace-Lorraine and the Polish Corridor)
- Military was severely restricted

This humiliation created deep resentment among Germans, creating fertile ground for political extremism.

### 2. The Rise of Totalitarian Regimes
Three aggressive dictatorships emerged with expansionist ambitions:
- **Germany (Nazi Germany):** Hitler sought to restore German pride and create *Lebensraum* ("living space") through conquest, particularly targeting Eastern Europe
- **Italy (Fascist Italy):** Mussolini aimed to revive a Roman Empire controlling the Mediterranean
- **Japan (Militarist Japan):*

## Tools

### Pre-built Tools

OpenAI offers a few built-in tools when using the OpenAIResponsesModel:

- The `WebSearchTool` lets an agent search the web.
- The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.
- The `ComputerTool` allows automating computer use tasks.
- The `CodeInterpreterTool` lets the LLM execute code in a sandboxed environment.
- The `HostedMCPTool` exposes a remote MCP server's tools to the model.
- The `ImageGenerationTool` generates images from a prompt.
- The `LocalShellTool` runs shell commands on your machine.

In [32]:
from agents import Agent, Runner, WebSearchTool

agent = Agent(
    name="Assistant",
    tools=[
        WebSearchTool(),
    ],
)

async def main():
    result = await Runner.run(agent, "Who is the current president of the United States as of 2026?")
    print(result.final_output)

await main()


As of January 6, 2026 the President of the United States is Donald J. Trump (47th President), inaugurated January 20, 2025. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Second_presidency_of_Donald_Trump?utm_source=openai))


### Custom Tools

You can define custom tools that the agent can use.

In [ ]:
import json
from typing_extensions import TypedDict, Any
from agents import Agent, FunctionTool, RunContextWrapper, function_tool


class Location(TypedDict):
    lat: float
    long: float

@function_tool  
async def fetch_weather(location: Location) -> str:
    
    """Fetch the weather for a given location.

    Args:
        location: The location to fetch the weather for.
    """
    # In real life, we'd fetch the weather from a weather API
    return f"The weather in {location['lat']}, {location['long']} is sunny"


@function_tool(name_override="fetch_data")  
def read_file(path: str, directory: str | None = None) -> str:
    """Read the contents of a file.

    Args:
        path: The path to the file to read.
        directory: The directory to read the file from.
    """
    # In real life, we'd read the file from the file system
    return "Hello, World!"


agent = Agent(
    name="Assistant",
    tools=[fetch_weather, read_file],  
    model=glm_model,
    model_settings=ModelSettings(include_usage=True),
)

result = await Runner.run(agent, "What is the weather in San Francisco?")
print(result.final_output)

result = await Runner.run(agent, "What are the contents of the file /tmp/hello.txt?")
print(result.final_output)

The weather in San Francisco is sunny!


ERROR:openai.agents:[non-fatal] Tracing: max retries reached, giving up on this batch.


The contents of the file `/tmp/hello.txt` are:

```
Hello, World!
```


ERROR:openai.agents:[non-fatal] Tracing: max retries reached, giving up on this batch.
ERROR:openai.agents:[non-fatal] Tracing: max retries reached, giving up on this batch.


## Sessions

In [30]:
from agents import Agent, Runner, SQLiteSession

# Create agent
agent = Agent(
    name="Assistant",
    instructions="Reply very concisely.",
    model=glm_model,
    model_settings=ModelSettings(include_usage=True)
)

# Create a new conversation
session = SQLiteSession(session_id="conv_123")

# Optionally resume a previous conversation by passing a conversation ID
# session = OpenAIConversationsSession(conversation_id="conv_123")

# Start conversation
result = await Runner.run(
    agent,
    "What city is the Golden Gate Bridge in?",
    session=session
)
print(result.final_output)  # "San Francisco"

# Continue the conversation
result = await Runner.run(
    agent,
    "What state is it in?",
    session=session
)
print(result.final_output)  # "California"

/usr/local/lib/python3.12/dist-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='San Fran...ion:** San Francisco.'}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


San Francisco.
California.


/usr/local/lib/python3.12/dist-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 6: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='Californ...swer:** "California."'}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


## Streaming

In [31]:
import asyncio
from openai.types.responses import ResponseTextDeltaEvent
from agents import Agent, Runner

async def main():
    agent = Agent(
        name="Joker",
        instructions="You are a helpful assistant.",
        model=glm_model,
        model_settings=ModelSettings(include_usage=True)
    )

    result = Runner.run_streamed(agent, input="Please tell me 5 jokes.")
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)



asyncio.run(main())

Here are 5 jokes for you:

1.  **I told my wife she was drawing her eyebrows too high.**
    She looked surprised.

2.  **Why don't skeletons fight each other?**
    They don't have the guts.

3.  **What did the ocean say to the beach?**
    Nothing, it just waved.

4.  **Why did the scarecrow win an award?**
    Because he was outstanding in his field.

5.  **I threw a boomerang a few years ago.**
    I now live in constant fear.

/usr/local/lib/python3.12/dist-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content="Here are...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
